In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
temps = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/temperature.npy')
clouds = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/cloud_cover.npy')
humidity = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/humidity.npy')
elevation = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/elevation.npy')
pressure = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/pressure.npy')
wind_dir = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/wind_dir.npy')
wind_speed = np.load('/content/drive/MyDrive/Colab Notebooks/хак yadro/wind_speed.npy')

In [ ]:
df = pd.DataFrame(data=[[hour, row, col, elevation[row][col], clouds[hour][row][col], humidity[hour][row][col],
                         pressure[hour][row][col], wind_dir[hour][row][col],
                         wind_speed[hour][row][col], temps[hour][row][col]]
                        for hour in range(43) for row in range(30) for col in range(30)],
                  columns=['hour', 'row', 'col', 'elevation', 'cloud', 'humidity', 'pressure', 'wind_dir', 'wind_speed', 'tempreture'])

df.columns = ['hour', 'row', 'col', 'elevation', 'cloud', 'humidity', 'pressure',
       'wind_dir', 'wind_speed', 'temperature']
df.tail(3)

In [ ]:
coordinates = [(i, j) for i in range(30) for j in range(30)]
x_values = [coord[0] for coord in coordinates[:900]]
y_values = [coord[1] for coord in coordinates[:900]]

In [ ]:
df_with_results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/хак yadro/solution_58_fix_temp_clouds1.csv')
df_with_results = pd.concat([pd.DataFrame(data = x_values * 5, columns = ['row']), pd.DataFrame(data = y_values * 5, columns = ['col']), df_with_results], axis = 1)

In [ ]:
df_with_results

In [ ]:
import math

# Функция для вычисления расстояния между двумя точками на карте
def distance(point1, point2):
    return math.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2) * 5

# Функция для поиска 9 ближайших соседей заданной точки
def find_nine_closest_neighbors(coordinates, target):
    neighbors = []
    min_neighbors = [(0, 0)] + [(0, 0)] * 8
    min_distances = [float('inf')] + [float('inf')] * 8

    for coord in coordinates:
        dist = distance(coord, target)
        if dist < max(min_distances):
            idx = min_distances.index(max(min_distances))
            min_distances[idx] = dist
            min_neighbors[idx] = coord

    for i in range(9):
        neighbors.append(min_neighbors[i])

    return neighbors

# Пример создания координат карты и заданных координат точки

target = (15, 15)

# Поиск координат девяти ближайших соседей для заданной точки
result = find_nine_closest_neighbors(coordinates, target)
print(result)


In [ ]:
target = (0, 0)
target_wind_dir = df[['hour', 'wind_dir']][(df['row'] == 0) & (df['col'] == 0)]
target_wind_dir.head(2)

In [ ]:
coordinates = [(i, j) for i in range(30) for j in range(30)]

def get_target_point_df(target_point, target_name, predictors, df):
  neighbors = find_nine_closest_neighbors(coordinates, target_point)
  wind_dir_predictors = ['wind_dir', 'wind_speed', 'pressure', 'tempreture']
  df_neighbors = pd.DataFrame()

  df = df


  for i, neig in enumerate(neighbors):
    names =[f'{i}_{predictor}' for predictor in predictors]

    if len(df_neighbors) == 0:
      df_neighbors = pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)
    else:
      df_neighbors = pd.concat([df_neighbors, pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)], axis = 1)

  # df_wind_speed = pd.concat([df_wind_speed.reset_index(), df['wind_speed'][(df['col'] == neig[0]) & (df['row'] == neig[1])].reset_index()], axis = 1, ignore_index=True, names=name)

  df_neighbors = pd.concat([df_neighbors, pd.DataFrame(df[target_name][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns=['target'])], axis = 1)
  # for col in df_neighbors.columns:
  #   if target_name in col:
  #     df_neighbors[col] = df_neighbors[col].shift(1)
  # df_neighbors = df_neighbors[1:]

  return df_neighbors

In [ ]:
df_neighbors = get_target_point_df(target_point = (0, 0), target_name = 'wind_speed', predictors = ['wind_dir', 'wind_speed', 'pressure', 'temperature', 'humidity'], df = df)
df_neighbors

In [ ]:
df_neighbors_results = get_target_point_df(target_point = (0, 0), target_name = 'wind_dir', predictors = ['wind_dir', 'wind_speed', 'pressure', 'temperature'], df = df_with_results)

In [ ]:
df_neighbors_results

# WIND DIR EXPIRIMENTS

In [ ]:
!pip install catboost

In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
from catboost import CatBoostRegressor
from IPython.display import display

In [ ]:
df_neighbors = pd.concat([df_neighbors, df_neighbors_results], axis = 0)

In [ ]:
df_neighbors.head()

In [ ]:
df_neighbors = get_target_point_df(target_point = (0, 0), target_name = 'wind_dir', predictors = predictors)
SPLIT_HOUR = 37
X = df_neighbors.drop(['target'], axis = 1)
y = df_neighbors.target

X_train, X_test, y_train, y_test = X.iloc[:SPLIT_HOUR].to_numpy(), X.iloc[SPLIT_HOUR].to_numpy(), y.iloc[:SPLIT_HOUR].to_numpy(), y.iloc[SPLIT_HOUR]
model = CatBoostRegressor(iterations=300, loss_function="RMSE", random_state=42)
model.fit(X_train, y_train, verbose=200)
pred = model.predict(X_test)
# mean_absolute_percentage_error(y_test, pred)
pred, y_test

In [ ]:
predictors = ['pressure', 'temperature', 'wind_dir']
pred = []
y_true = []
SPLIT_HOUR = 41
for point in coordinates:
  df_train = get_target_point_df(target_point = point, target_name = 'wind_dir', predictors = predictors, df = df)

  X = df_train.drop(['target'], axis = 1)
  y = df_train.target

  X_train, X_test, y_train, y_test = X.iloc[:SPLIT_HOUR].to_numpy(), X.iloc[SPLIT_HOUR].to_numpy(), y.iloc[:SPLIT_HOUR].to_numpy(), y[SPLIT_HOUR]
  model = CatBoostRegressor(iterations=400, loss_function="RMSE", random_state=42)
  model.fit(X_train, y_train, verbose=200)
  pred.append(model.predict(X_test))
  # print(model.predict(X_test))
  y_true.append(y_test)

In [ ]:
mape_lst = []
mape_1 = mean_absolute_percentage_error(y_true, pred)
pred_1 = pred
# np.array(pred).shapeч
mape_lst.append(mape_1)

In [ ]:
def predict_feature_with_catbooster(feature_name, df_to_refactor, predictors):
  y_predict = []
  for point in coordinates:
    df_train = get_target_point_df(target_point = point, target_name = feature_name, predictors = predictors, df = df_to_refactor)
    X = df_train.drop(['target'], axis = 1)
    y = df_train.target

    X_train, y_train = X.to_numpy(), y.to_numpy()
    model = CatBoostRegressor(iterations=400, loss_function="RMSE", random_state=42)
    model.fit(X_train, y_train, verbose=200)
    y_predict.append(model.predict(X_test))

  return y_predict

In [ ]:
coordinates = [(i, j) for i in range(30) for j in range(30)]
x_values = [coord[0] for coord in coordinates[:900]]
y_values = [coord[1] for coord in coordinates[:900]]

df = pd.DataFrame(data=[[hour, row, col, elevation[row][col], clouds[hour][row][col], humidity[hour][row][col],
                         pressure[hour][row][col], wind_dir[hour][row][col],
                         wind_speed[hour][row][col], temps[hour][row][col]]
                        for hour in range(43) for row in range(30) for col in range(30)],
                  columns=['hour', 'row', 'col', 'elevation', 'cloud', 'humidity', 'pressure', 'wind_dir', 'wind_speed', 'tempreture'])

df.columns = ['hour', 'row', 'col', 'elevation', 'cloud_cover', 'humidity', 'pressure',
       'wind_dir', 'wind_speed', 'temperature']

df_with_results = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/хак yadro/const_wind_dir (13).csv').drop(['ID'], axis = 1)
df_with_results = pd.concat([pd.DataFrame(data = x_values * 5, columns = ['row']), pd.DataFrame(data = y_values * 5, columns = ['col']), df_with_results], axis = 1).reset_index()

df_overall = pd.concat([df, df_with_results], axis = 0).reset_index().drop(['index'], axis = 1)
# df_overall.tail()
df_overall.iloc[:38700 + 900 * 5]

In [ ]:
for hour in range(5):

  wind_dir_pred = predict_feature_with_catbooster(feature_name = 'wind_dir', df_to_refactor=df_overall.iloc[:38700 + 900 * hour], predictors = ['pressure', 'temperature', 'wind_dir'])

  df_overall['wind_dir'].iloc[38700 + 900 * hour: 38700 + 900 * (hour + 1)] = wind_dir_pred

  print()
  print('YES', hour)
  print()

In [ ]:
df_overall

In [ ]:
df_1_hour_above = pd.concat([df, df_with_results], axis = 0).reset_index()

In [ ]:
# начинать с 38700
df_1_hour_above = pd.concat([df, df_with_results], axis = 0).reset_index()
ONE_HOUR_ABOVE = 38700
POINTS = 900
df_1_hour_above['wind_dir'].iloc[38700: ONE_HOUR_ABOVE + POINTS] = pred_2

In [ ]:
SPLIT_HOUR = 43
pred_3 = []
for point in coordinates:
  df_train = get_target_point_df(target_point = point, target_name = 'wind_dir', predictors = predictors, df = df_1_hour_above)

  X = df_train.drop(['target'], axis = 1)
  y = df_train.target

  X_train, y_train = X.to_numpy(), y.to_numpy()
  model = CatBoostRegressor(iterations=400, loss_function="RMSE", random_state=42)
  model.fit(X_train, y_train, verbose=200)
  pred_3.append(model.predict(X_test))

In [ ]:
  df_1_hour_above['wind_dir'].iloc[38700 + POINTS: ONE_HOUR_ABOVE + 2*POINTS] = pred_3

  SPLIT_HOUR = 44
  pred_4 = []
  for point in coordinates:
    df_train = get_target_point_df(target_point = point, target_name = 'wind_dir', predictors = predictors, df = df_1_hour_above)

    X = df_train.drop(['target'], axis = 1)
    y = df_train.target

    X_train, y_train = X.to_numpy(), y.to_numpy()
    model = CatBoostRegressor(iterations=400, loss_function="RMSE", random_state=42)
    model.fit(X_train, y_train, verbose=200)
    pred_2.append(model.predict(X_test))

In [ ]:
np.array(pred).flatten()[:10]

In [ ]:
predictors = ['wind_dir', 'wind_speed', 'pressure', 'tempreture']
mape_lst = []
for point in coordinates:
  df_neighbors = get_target_point_df(target_point = point, target_name = 'wind_dir', predictors = predictors)
  SPLIT_HOUR = 40
  X = df_neighbors.drop(['target'], axis = 1)
  y = df_neighbors.target

  X_train, X_test, y_train, y_test = X.iloc[:SPLIT_HOUR].to_numpy(), X.iloc[SPLIT_HOUR:].to_numpy(), y.iloc[:SPLIT_HOUR].to_numpy(), y.iloc[SPLIT_HOUR:].to_numpy()
  model = CatBoostRegressor(iterations=300, loss_function="RMSE", random_state=42)
  model.fit(X_train, y_train, verbose=200)
  pred = model.predict(X_test)
  mape_lst.append(mean_absolute_percentage_error(y_test, pred))

In [ ]:
df_with_results['wind_dir'] = pd.Series(data = np.array(pred).flatten())

In [ ]:
last_wind_dir = df_overall['wind_dir'].iloc[38700:]
list(last_wind_dir)

In [ ]:
df_with_results['wind_dir'] = list(last_wind_dir)
df_with_results['ID'] = range(4500)
df_with_results

In [ ]:
df_with_results[['ID', 'temperature', 'pressure', 'humidity', 'wind_speed', 'wind_dir', 'cloud_cover']].to_csv("/content/drive/MyDrive/Colab Notebooks/хак yadro/best_with_changed_wind_dir.csv", index=False)

In [ ]:
pip install optuna

In [ ]:
from sklearn.model_selection import GridSearchCV
import optuna
from sklearn.model_selection import train_test_split

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8],
    'iterations': [100, 200, 300]
}

predictors = ['humidity', 'wind_speed', 'temperature', 'humidity']

df_neighbors = get_target_point_df(target_point = (15, 15), target_name = 'wind_speed', predictors = predictors, df=df)
SPLIT_HOUR = 41
X = df_neighbors.drop(['target'], axis = 1)
y = df_neighbors.target

# X_train, X_test, y_train, y_test = X.iloc[:SPLIT_HOUR].to_numpy(), X.iloc[SPLIT_HOUR:].to_numpy(), y.iloc[:SPLIT_HOUR].to_numpy(), y.iloc[SPLIT_HOUR:].to_numpy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1, random_state = 42)
# model = CatBoostRegressor(iterations=300, loss_function="RMSE", random_state=42)
# model = CatBoostRegressor
# model.fit(X_train, y_train, verbose=200)

# Обучение модели с использованием Grid Search
# grid_search.fit(X_train, y_train)

# pred = model.predict(X_test)
# mape_lst.append(mean_absolute_percentage_error(y_test, pred))def objective(trial):
# def objective(trial):
#     params = {
#         'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.2),
#         'depth': trial.suggest_int('depth', 4, 10),
#         'iterations': trial.suggest_int('iterations', 100, 500),
#         'random_seed': 42
#     }

#     model = CatBoostRegressor(**params)
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)
#     mape = mean_absolute_percentage_error(y_test, y_pred)

#     return mape

# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)



In [ ]:
best_par = study.best_params

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.11, random_state = 42)
X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

In [ ]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_uniform('learning_rate', 0.01, 0.2),
        'depth': trial.suggest_int('depth', 4, 10),
        'iterations': trial.suggest_int('iterations', 100, 500),
        'random_seed': 42
    }

    model = CatBoostRegressor(**params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return mape

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
study.best_params, study.best_value


In [ ]:
pred = []
pred_mapes = []
y_testing = []
for poin in coordinates:
  train_data = get_target_point_df(target_name = 'wind_speed', target_point= point, df = df, predictors = ['humidity', 'wind_dir', 'wind_speed', 'pressure', 'temperature'])
  X = train_data.drop(['target'], axis = 1)
  y = train_data['target']
  model = CatBoostRegressor(learning_rate = 0.011062607084344443, depth = 4, iterations = 440, random_state = 42)

  X_train, X_test, y_train, y_test = train_data.drop(['target'], axis = 1).iloc[:37], train_data.drop(['target'], axis = 1).iloc[37:], train_data['target'].iloc[:37], train_data['target'].iloc[37:]
  X_train, X_test, y_train, y_test = X_train.to_numpy(), X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

  model.fit(X_train, y_train)
  pred.append(model.predict(X_test))
  y_testing.append(y_test)
  pred_mapes.append()


In [ ]:
mean_absolute_percentage_error(pred, y_testing)

In [ ]:
prediction = []
for poin in coordinates:
  train_data = get_target_point_df(target_name = 'wind_speed', target_point= point, df = df, predictors = ['humidity', 'wind_dir', 'wind_speed', 'pressure', 'temperature'])
  model = CatBoostRegressor(learning_rate = 0.011062607084344443, depth = 4, iterations = 440, random_state = 42)

  X_test = train_data.drop(['target'], axis = 1).iloc[5:].to_numpy()

  pred.append(model.predict(X_test))

# BOOST MODELS

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_percentage_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier, HistGradientBoostingRegressor
from mlxtend.plotting import plot_decision_regions

In [ ]:
class GBMRegressor:
    def __init__(self, learning_rate=0.1, n_estimators=100, max_depth=3, random_state=0):
        self.learning_rate = learning_rate
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.random_state = random_state
        self.trees = []

    def fit(self, X, y):
        self.initial_leaf = y.mean()
        predictions = np.zeros(len(y)) + self.initial_leaf

        for _ in range(self.n_estimators):
            residuals = y - predictions
            tree = DecisionTreeRegressor(criterion='friedman_mse', max_depth=self.max_depth,
                                         random_state=self.random_state)
            tree.fit(X, residuals)
            predictions += self.learning_rate * tree.predict(X)
            self.trees.append(tree)

    def predict(self, samples):
        predictions = np.zeros(len(samples)) + self.initial_leaf

        for i in range(self.n_estimators):
            predictions += self.learning_rate * self.trees[i].predict(samples)

        return predictions

# WIND SPEED EXPERIMENTS

In [ ]:
coordinates = [(i, j) for i in range(30) for j in range(30)]

def get_target_point_df(target_point, target_name, predictors, df, shift = False):
  neighbors = find_nine_closest_neighbors(coordinates, target_point)
  wind_dir_predictors = ['wind_dir', 'wind_speed', 'pressure', 'tempreture']
  df_neighbors = pd.DataFrame()

  df = df

  for i, neig in enumerate(neighbors):
    names =[f'{i}_{predictor}' for predictor in predictors]

    if len(df_neighbors) == 0:
      df_neighbors = pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)
    else:
      df_neighbors = pd.concat([df_neighbors, pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)], axis = 1)

  df_neighbors = pd.concat([df_neighbors, pd.DataFrame(df[target_name][(df['col'] == target_point[0]) & (df['row'] == target_point[1])].to_numpy(), columns=['target'])], axis = 1)

  if shift:
    for col in df_neighbors.columns:
      if target_name in col:
        df_neighbors[col] = df_neighbors[col].shift(1)
    df_neighbors = df_neighbors[1:]

  return df_neighbors

In [ ]:
def double_exponential_smoothing(series, alpha, beta):
    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # прогнозируем
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha*value + (1-alpha)*(level+trend)
        trend = beta*(level-last_level) + (1-beta)*trend
        result.append(level+trend)
    return result

def exponential_smoothing(series, alpha):
    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return np.array(result)

In [ ]:
TARGET_NAME = 'wind_speed'
DF = df
PREDICTORS = ['wind_speed', 'humidity', 'temperature', 'pressure']

train_df = get_target_point_df(target_name = TARGET_NAME, df = DF, predictors = PREDICTORS, target_point = (0, 0), shift = True)

y = double_exponential_smoothing(train_df.target.values, 0.03, 0.5)

In [ ]:
# test diff alpha and beta

def predict_target(points, alpha, model, beta = None ):
  prediction_lst = []
  y_test_lst = []

  for point in points:
    train_df = get_target_point_df(target_name = TARGET_NAME, df = DF, predictors = PREDICTORS, target_point = point, shift = True)
    X = train_df.drop(['target'], axis = 1).to_numpy()
    y = exponential_smoothing(series = train_df['target'].values, alpha = alpha)
    X_train, X_test, y_train, y_test = X[:41], [X[41]], y[:41], y[41]

    model = model
    model.fit(X_train, y_train)
    prediction = model.predict(
        X_test
    )
    prediction_lst.append(prediction)
    y_test_lst.append(train_df['target'].iloc[41])

  return prediction_lst, y_test_lst

In [ ]:
pip install catboost

In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
mape_min = 100
for alpha in [0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3]:
  pred, y_true = predict_target(points = coordinates[:150], alpha = alpha, beta = 0.19, model = XGBRegressor())
  mape = mean_absolute_percentage_error(pred, y_true)

  if mape_min > mape:
    mape_min = mape
    print(alpha, mape_min)

In [ ]:
mape_min = 100
for alpha in [0.15, 0.16, 0.17, 0.18, 0.19, 0.20, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3]:
  pred, y_true = predict_target(points = coordinates[:150], alpha = alpha, beta = 0.19, model = XGBRegressor(max_depth=4, random_state=3))
  mape = mean_absolute_percentage_error(pred, y_true)

  if mape_min > mape:
    mape_min = mape
    print(alpha, mape_min)

In [ ]:
pred, y_true = predict_target(alpha = 0.32, model = XGBRegressor(random_state = 3, max_depth = 4), points = coordinates)

In [ ]:
mean_absolute_percentage_error(y_true, pred)

In [ ]:
pred, y_true = predict_target(alpha = 0.32, model = CatBoostRegressor(iterations=700, max_depth=5, verbose=False, random_state=3), points = coordinates)

In [ ]:
mean_absolute_percentage_error(y_true, pred)

In [ ]:
pred, y_true = predict_target(alpha = 0.32, model = CatBoostRegressor(iterations=300, max_depth=5, verbose=False), points = coordinates)

In [ ]:
mean_absolute_percentage_error(y_true, pred)

# MLForest TESTING


In [ ]:
pip install mlforecast

In [ ]:
import random
import tempfile

import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb

from mlforecast.lag_transforms import ExpandingMean, ExponentiallyWeightedMean, RollingMean
from mlforecast.lgb_cv import LightGBMCV
from mlforecast.target_transforms import Differences, LocalStandardScaler
from mlforecast.utils import generate_daily_series
from mlforecast import MLForecast

In [ ]:
fcst = MLForecast(
    models=lgb.LGBMRegressor(random_state=0, verbosity=-1),
    freq=1,
    lags=[24 * (i+1) for i in range(7)],
    lag_transforms={
        48: [ExponentiallyWeightedMean(alpha=0.3)],
    },
    num_threads=1,
    target_transforms=[Differences([24])],
)

In [ ]:
df_for_training = get_target_point_df(target_name = 'wind_speed', target_point = (0, 0), predictors = PREDICTORS, shift = True, df = DF)
HOUR = 41
X_train, X_test, y_train, y_test = df_for_training.drop(['target'], axis = 1).iloc[:HOUR], df_for_training.drop(['target'], axis = 1).iloc[HOUR], df_for_training['target'].iloc[:HOUR], df_for_training['target'].iloc[HOUR]

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn.model_selection import train_test_split,cross_val_score, cross_val_predict
import missingno as msno # plotting missing data
import seaborn as sns # plotting library
from sklearn import svm

# SIMPLE MODELS TESTING

In [ ]:
lr = LinearRegression(n_jobs = -1)

In [ ]:
pred, y_test = predict_target(alpha = 0.32, model = lr, points = coordinates)

In [ ]:
mean_absolute_percentage_error(y_test, pred)

In [ ]:
dr = DecisionTreeRegressor(random_state=2)
pred, y_test = predict_target(alpha = 0.32, model = dr, points = coordinates)
mean_absolute_percentage_error(y_test, pred)

In [ ]:
rf = RandomForestRegressor(n_jobs=-1, max_depth = 4, random_state = 3)
pred, y_test = predict_target(alpha = 0.32, model = rf, points = coordinates)
mean_absolute_percentage_error(y_test, pred)

In [ ]:
from sklearn.svm import SVR

In [ ]:
regressor= SVR(kernel='rbf')
pred, y_test = predict_target(alpha = 0.32, model = regressor, points = coordinates)
mean_absolute_percentage_error(y_test, pred)

In [ ]:
def get_X_y(points):
  X = []
  y = []

  for point in points:
    train_df = get_target_point_df(target_name = TARGET_NAME, df = DF, predictors = PREDICTORS, target_point = point, shift = True)
    X.append(train_df.drop(['target'], axis = 1).to_numpy())
    y.append(train_df['target'].to_numpy())

  return X, y

In [ ]:
np.array(X).shape

In [ ]:
X, y = get_X_y(coordinates)

In [ ]:
X_train = np.array(X)[:, :-1, :]
X_test = np.array(X)[:, -1, :]
y_train = np.array(y)[:, :-1]
y_test = np.array(y)[:, -1]

# NEW TRIALS

In [ ]:
coordinates = [(i, j) for i in range(30) for j in range(30)]

def get_target_point_df(target_point, target_name, predictors, df, shift = False):
  neighbors = find_nine_closest_neighbors(coordinates, target_point)
  wind_dir_predictors = ['wind_speed', 'pressure', 'tempreture', 'humidity']
  df_neighbors = pd.DataFrame()

  df = df

  for i, neig in enumerate(neighbors):
    names =[f'{i}_{predictor}' for predictor in predictors]

    if len(df_neighbors) == 0:
      df_neighbors = pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)
    else:
      df_neighbors = pd.concat([df_neighbors, pd.DataFrame(data = df[predictors][(df['col'] == neig[0]) & (df['row'] == neig[1])].to_numpy(), columns = names)], axis = 1)

  df_neighbors = pd.concat([df_neighbors, pd.DataFrame(df[target_name][(df['col'] == target_point[0]) & (df['row'] == target_point[1])].to_numpy(), columns=['target'])], axis = 1)

  if shift:
    for col in df_neighbors.columns:
      if target_name in col:
        df_neighbors[col] = df_neighbors[col].shift(1)
    df_neighbors = df_neighbors[1:]

  return df_neighbors

In [ ]:
def predict_target(points, alpha, model, beta = None ):
  prediction_lst = []
  y_test_lst = []

  for point in points:
    train_df = get_target_point_df(target_name = TARGET_NAME, df = DF, predictors = PREDICTORS, target_point = point, shift = True)
    X = train_df.drop(['target'], axis = 1).to_numpy()
    y = exponential_smoothing(series = train_df['target'].values, alpha = alpha)
    X_train, X_test, y_train, y_test = X[:41], [X[41]], y[:41], y[41]

    model = model
    model.fit(X_train, y_train)
    prediction = model.predict(
        X_test
    )
    prediction_lst.append(prediction)
    y_test_lst.append(train_df['target'].iloc[41])

  return prediction_lst, y_test_lst

In [ ]:
pred, y_test = predict_target(alpha = 0.32, points = coordinates, model = XGBRegressor(max_depth=4, n_estimators=500))

In [ ]:
mean_absolute_percentage_error(y_test, pred)

In [ ]:
from catboost import Pool, cv
cat = CatBoostRegressor(iterations = 400, depth=4)

one_point_df = get_target_point_df(target_point = (15, 15), target_name = 'wind_speed', predictors = ['humidity', 'wind_speed', 'pressure', 'temperature'], shift = True, df = DF)
X_train, X_test, y_train, y_test = one_point_df.drop(['target'], axis = 1).iloc[:41], one_point_df.drop(['target'], axis = 1).iloc[41], one_point_df['target'][:41], one_point_df['target'][41]

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_search_result = cat.grid_search(grid, X=X_train.to_numpy(), y=y_train.to_numpy(), plot=True)

In [ ]:
pred, y_test = predict_target(alpha = 0.32, points = coordinates, model = XGBRegressor(max_depth=4, n_estimators=397))

In [ ]:
mean_absolute_percentage_error(y_test, pred)